# SQLite DB: Storing and Retrieving Vectors

In [1]:
import sqlite3
import numpy as np

In [ ]:
# create a connection to the SQLite DB
conn = sqlite3.connect('sqlite/vectors.db')
# Create a cursor object to execute SQL Commands
cursor = conn.cursor()

In [11]:
# Create a table for vector data
cursor.execute(
"""
CREATE TABLE IF NOT EXISTS vectors (
    id INTEGER PRIMARY KEY,
    vector BLOB NOT NULL
)
"""
)

In [12]:
# generate some sample vectors
vect1 = np.array([1.2, 3.4, 2.1, 0.8])
vect2 = np.array([2.7, 1.5, 3.9, 2.3])

In [13]:
vect1.tobytes() # numpy array to bytestream

b'333333\xf3?333333\x0b@\xcd\xcc\xcc\xcc\xcc\xcc\x00@\x9a\x99\x99\x99\x99\x99\xe9?'

In [14]:
# Insert vector data into table
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", 
               (sqlite3.Binary(vect1.tobytes()),))


In [15]:
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", 
               (sqlite3.Binary(vect2.tobytes()),))

In [16]:
# Retreive data

cursor.execute("SELECT vector FROM vectors")

In [17]:
rows = cursor.fetchall()

In [18]:
rows

[(b'333333\xf3?333333\x0b@\xcd\xcc\xcc\xcc\xcc\xcc\x00@\x9a\x99\x99\x99\x99\x99\xe9?',),
 (b'\x9a\x99\x99\x99\x99\x99\x05@\x00\x00\x00\x00\x00\x00\xf8?333333\x0f@ffffff\x02@',)]

In [19]:
vector = np.frombuffer(rows[0][0], dtype = np.float64)

In [20]:
vector

array([1.2, 3.4, 2.1, 0.8])

In [21]:
vectors = []
for row in rows:
    vector = np.frombuffer(row[0], dtype = np.float64)
    vectors.append(vector)

In [22]:
vectors

[array([1.2, 3.4, 2.1, 0.8]), array([2.7, 1.5, 3.9, 2.3])]

# Vector Similarity Search (VSS)

In [23]:
query_vect = np.array([1.0, 3.2, 2.0, 0.5]) 

In [24]:
cursor.execute("""
SELECT vector FROM vectors ORDER BY abs(vector - ?) ASC
""", (sqlite3.Binary(query_vect.tobytes()),))

In [25]:
res = cursor.fetchone()  # finding the top one

In [26]:
np.frombuffer(res[0], dtype=np.float64) # most similar vector

array([2.7, 1.5, 3.9, 2.3])

In [27]:
conn.commit()

In [28]:
conn.close()